# Data preparation

In [1]:
URL = 'https://nnfs.io/datasets/fashion_mnist_images.zip'
FILE = 'fashion_mnist_images.zip'
FOLDER = 'examples/fashion_mnist/data/'

In [2]:
import os
import urllib
import urllib.request

In [3]:
if not os.path.isfile(FILE):
    print(f"Downloading {URL} and saving as {FILE}...")
    urllib.request.urlretrieve(URL,FILE)

In [4]:
from zipfile import ZipFile

print("Unzipping images...")
with ZipFile(FILE) as zip_images:
    zip_images.extractall(FOLDER)

Unzipping images...


# Data loading

In [10]:
import os

DATA_DIR = 'examples/fashion_mnist/data'
TRAIN_DIR = 'examples/fashion_mnist/data/train'
TEST_DIR = 'examples/fashion_mnist/data/test'

labels = os.listdir(TRAIN_DIR)

files = os.listdir(TRAIN_DIR + '/0')

In [11]:
import cv2
import numpy as np
image_data = cv2.imread(TRAIN_DIR + '/7/0002.png', cv2.IMREAD_UNCHANGED)

np.set_printoptions(linewidth=200)

In [13]:
def load_mnist_dataset(dataset,path):
    
    # Scan ll the directories and create a list of labels
    labels = os.listdir(os.path.join(path, dataset))
    
    X, y = [], []
    
    for label in labels:
        for file in os.listdir(os.path.join(path, dataset, label)):
            image = cv2.imread(os.path.join(path, dataset, label, file), cv2.IMREAD_UNCHANGED)
        
            X.append(image)
            y.append(label)
            
    # Convert the data to proper numpy arrays and return
    return np.array(X), np.array(y).astype('uint8')

def create_data_mnist(path):
    # Load both sets separately
    X, y = load_mnist_dataset('train', path)
    X_test, y_test = load_mnist_dataset('test', path)
    # And return all the data
    return X, y, X_test, y_test

X, y, X_test, y_test = create_data_mnist(DATA_DIR)

# Data preprocessing

In [14]:
# Scale features between -1 and 1
X = (X.astype(np.float32) - 127.5) / 127.5
X_test = (X_test.astype(np.float32) - 127.5) / 127.5
assert -1 <= X.min(), X.max() <= 1 # Ensuring that scaling is correct

In [15]:
X.shape

(60000, 28, 28)

In [16]:
X = X.reshape(60000, -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Data shuffling

In [22]:
keys = np.array( range (X.shape[ 0 ]))
import nnfs
nnfs.init()
np.random.shuffle(keys)

# Training

In [24]:
X,y,X_test,y_test = create_data_mnist(DATA_DIR)

In [25]:
# shuffle
keys = np.array(range(X.shape[0]))
np.random.shuffle(keys)
X = X[keys]
y = y[keys]

In [26]:
# flatten for dense NN, from -1 to 1
X = (X.reshape(X.shape[ 0 ], - 1 ).astype(np.float32) - 127.5 ) / 127.5
X_test = (X_test.reshape(X_test.shape[ 0 ], - 1 ).astype(np.float32) -
127.5 ) / 127.5

In [32]:
from nn.model import *

from nn.layer import *
from nn.activation import *

from nn.optimizer import *
from nn.loss import *
from nn.accuracy import *

# Instantiate the model
model = Model()
# Add layers
model.add(Layer_Dense(X.shape[ 1 ], 64 ))
model.add(Activation_ReLU())
model.add(Layer_Dense( 64 , 64 ))
model.add(Activation_ReLU())
model.add(Layer_Dense( 64 , 10 ))
model.add(Activation_Softmax())

# Set loss, optimizer and accuracy objects
model.set(
    loss = Loss_CategoricalCrossentropy(),
    optimizer = Optimizer_Adam( decay = 5e-5 ),
    accuracy = Accuracy_Categorical()
)

# Finalize the model
model.finalize()
# Train the model
model.train(X, y, validation_data = (X_test, y_test), 
            epochs = 5 , batch_size = 128 , print_every = 100)

epoch: 1
step: 0, acc: 0.227, loss: 2.302 (data_loss: 2.302, reg_loss: 0.000), lr: 0.001
step: 100, acc: 0.602, loss: 0.943 (data_loss: 0.943, reg_loss: 0.000), lr: 0.0009950248756218907
step: 200, acc: 0.703, loss: 0.754 (data_loss: 0.754, reg_loss: 0.000), lr: 0.0009900990099009901
step: 300, acc: 0.805, loss: 0.550 (data_loss: 0.550, reg_loss: 0.000), lr: 0.0009852216748768474
step: 400, acc: 0.766, loss: 0.568 (data_loss: 0.568, reg_loss: 0.000), lr: 0.000980392156862745
step: 468, acc: 0.854, loss: 0.488 (data_loss: 0.488, reg_loss: 0.000), lr: 0.0009771350400625367
training, acc: 0.722, loss: 0.742 (data_loss: 0.742, reg_loss: 0.000), lr: 0.0009771350400625367
validation, acc: 0.806, loss: 0.533
epoch: 2
step: 0, acc: 0.789, loss: 0.590 (data_loss: 0.590, reg_loss: 0.000), lr: 0.0009770873027505008
step: 100, acc: 0.812, loss: 0.580 (data_loss: 0.580, reg_loss: 0.000), lr: 0.000972337012008362
step: 200, acc: 0.773, loss: 0.542 (data_loss: 0.542, reg_loss: 0.000), lr: 0.000967632